In [1]:
### Deployment ###
# assumption: incoming data is raw data as in the training set, but it comes without the label columns, it comes without missing values?
# 

In [2]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [4]:
# load train and test data again
body = client_3b08f636aa4c4bc58b03e36105622462.get_object(Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='online_shoppers_intention_etl.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df = pd.read_csv(body)
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.name = 'select columns'
        self.columns = columns
    
    def fit( self, X, y = None ):
        return self 
    
    def transform(self, X, y = None):
        return X[self.columns] 

class NumericalTransformer(BaseEstimator, TransformerMixin):

    def __init__( self):
        self.name = 'numerical features'
        
    def fit(self, X, y = None):
        return self 

    def transform(self, X, y = None):
        #X = self.scale_features(df=X, columns=self.cols_scaling)
        # print('num features', X.shape)
        # return X.values
        return self
    
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    
    def __init__( self):
        self.name = 'categorical features'
    
    def fit(self, X, y = None):
        return self 

    def transform(self, X, y = None):
        # X = self.get_binary_weekend(df=X)
        # print('cat features', X.shape)
        # return X.values
        return self

In [8]:
cat_cols = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']
num_cols = ['Administrative',
             'Administrative_Duration',
             'Informational',
             'Informational_Duration',
             'ProductRelated',
             'ProductRelated_Duration',
             'BounceRates',
             'ExitRates',
             'PageValues',
             'SpecialDay']

#( 'cat_transformer', CategoricalTransformer() ), ( 'num_transformer', NumericalTransformer() )

categorical_pipeline = Pipeline( steps = [( 'one_hot_encoder', OneHotEncoder() )] )
      
numerical_pipeline = Pipeline( steps = [ ( 'standardscaler', StandardScaler() )] )

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, num_cols),
        ('cat', categorical_pipeline, cat_cols)])

# use the model that was created in the modelling notebook
full_pipeline_m = Pipeline( steps = [ ( 'preprocessor', preprocessor),
                                  
                                  ( 'model', GradientBoostingClassifier() ) ] )

# due to the slightly different preprocessing i need to train the model again but i will iuse the same paarmeters as i used for the best model
# in the modelling notebook

model = full_pipeline_m.fit(X=X_train, y=y_train)

In [9]:
#Can predict with it like any other pipeline
y_pred = full_pipeline_m.predict(X=X_test)

In [10]:
full_pipeline_m.score(X=X_train, y=y_train), full_pipeline_m.score(X=X_test, y=y_test)

(0.9239537534603485, 0.9045913383262781)

In [11]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import pickle

wml_credentials={
  "url": "https://eu-gb.ml.cloud.ibm.com",
  "apikey": "jIs1rqQ818pgStOECwxmvhMxsWLyskFBzTHjfzWi76ET",
  "username": "Service credentials-1",
  "instance_id": "f45142b5-b178-444d-b30b-494230b95138"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

2020-05-11 14:35:51,923 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2020-05-11 14:36:12,924 - watson_machine_learning_client.wml_client_error - WARNING - Deployment creation failed. Error: 402. {"trace":"1m8zeccdlsibb","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}


In [12]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Tjark Petersen", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "tjark.petersen@ibm.com", 
               client.repository.ModelMetaNames.NAME: "shopping_behaviour_final_model"}

model_artifact=client.repository.store_model(model, meta_props=model_props, training_data=X_train, training_target=y_train)

In [22]:
#Get model UID
published_model_uid = client.repository.get_model_uid(model_artifact)
#Deploy the model
created_deployment = client.deployments.create(published_model_uid, name="shopping_behaviour_deployment")



#######################################################################################

Synchronous deployment creation for uid: 'a291d7fb-068c-416a-a5d1-68be647ee787' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6d9f7dc2-dbc0-4124-89e8-bfeb78129ebf'
------------------------------------------------------------------------------------------------




In [23]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
print(scoring_endpoint)

https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/f45142b5-b178-444d-b30b-494230b95138/deployments/6d9f7dc2-dbc0-4124-89e8-bfeb78129ebf/online


In [36]:
# example input

example_values = [str(k) for k in X_test.iloc[22,:].values]
scoring_payload = {"fields": list(X_test.columns),"values":[example_values]}
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print(predictions)

{'fields': ['prediction', 'probability'], 'values': [[1, [0.29195858967648536, 0.7080414103235146]]]}


In [21]:
client.deployments.list()

----  ----  ----  -----  -------  ---------  -------------
GUID  NAME  TYPE  STATE  CREATED  FRAMEWORK  ARTIFACT TYPE
----  ----  ----  -----  -------  ---------  -------------


In [20]:
client.repository.delete('2ca3d267-e418-4f51-9902-455a6ddafd8d')

'SUCCESS'